In [5]:
import torch
import model
from tokenizer import BytePairTokenizer, TikToken4o

In [11]:
phrase = 'every journey begins with a single step'

tokenizer = TikToken4o
tokens = tokenizer.encode(phrase)
embeddings = model.embedding_layer(tokens)

AttributeError: module 'model' has no attribute 'embedding_layer'

In [10]:
tokens

[50492, 12647, 18015, 483, 261, 4590, 5983]